# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Utilizza il seguente codice come punto di partenza, riempiendo gli spazi dove manca codice!


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [1]:
# 1. Registra gli ordini in una lista di classi Order

GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

orders = [] # Lista di oggetti che descrivono gli ordini ricevuti

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste
        # Ipotizzo che la struttura mail sia tale da avere il numero di maglie che segue sempre la parola maglietta/e nell aprima riga
        self.quantity = int(lines[1].split(" ")[lines[1].split(" ").index('magliette') -1])
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente (ultima riga del txt)
        self.customer =f"{lines[len(lines)-1].split(' ')[0]}  {lines[len(lines)-1].split(' ')[1]}" #salvo nome e cognome come stringa
        
        # Salva il prezzo richiesto dal cliente (bid price)
        self.bid_price = float(lines[3].split(' ')[-1][:-1]) # non prendo il simbolo €
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        if self.bid_price < ask_price:
            return 0
        else:
            return ask_price
    
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, cost_per_km = 0):
        costumer_cash=self.Payment(ask_price)
        if costumer_cash ==0:
            return 0
        else:
            global production_cost
            profit=(ask_price-production_cost)*self.quantity #ovviamente non vendo se profitto è negativo
            if profit>0:
                return profit
            else:
                return 0
    
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return (((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5
    
     # calcolo costi di trasporto per agevolarmi nella creazione succesiva della lista profitti
    def costo_trasporto(self):
        if self.quantity<100:
            return (0.1*self.quantity*self.Distance(GCS_Piazza_Leo))
        else:
            return (0.05*self.quantity*self.Distance(GCS_Piazza_Leo))
    
    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
    
# Crea gli oggetti Order e li registra nella lista orders
for i in range(num_orders):
    file_name=f'{i+1}.txt'
    file=open(file_name,'r')
    orders.append(Order(file.read()))
    file.close()

In [2]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti  
possible_profit=[[sum(orders[j].Profit(orders[i].bid_price,0) for j in range(num_orders))]for i in range(num_orders)]
##for pp in possible_profit:
   ## print(pp)

# Determina il profitto maggiore
max_profit = max(possible_profit)

# Determina il prezzo migliore
best_price = orders[possible_profit.index(max_profit)].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")

Il miglior prezzo è €13.22
Si eseguiranno 195 ordini


In [3]:
# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.

cost_less_100 = 0.10
cost_at_least_100 = 0.05
#si potrebbe includere il calcolo del costo di trasporto e il suo effetto direttamente nella funzione Profit o in una funzione ausiliaria

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits_2 = [[sum((orders[j].Profit(orders[i].bid_price,0)*orders[j].quantity-orders[j].costo_trasporto()) for j in range(num_orders))]for i in range(num_orders)]

# Determina il profitto maggiore
max_profitto = max(possible_profits_2)

# Determina il prezzo migliore
best_price = orders[possible_profits_2.index(max_profitto)].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ordine for ordine in orders if (ordine.Profit(best_price)-ordine.costo_trasporto())>0]
print("Si eseguiranno", len(orders_to_go),"ordini")

######## SCRIVI CODICE ########
# ... assembla la lista degli ordini che devono che si possono soddisfare, dando precedenza a quelli che portano maggiore profitto
######## SCRIVI CODICE ########
for i in range(len(orders_to_go)-1):
    for j in range((i+1),len(orders_to_go)):
        if (orders_to_go[i].Profit(best_price)-orders_to_go[i].costo_trasporto())>(orders_to_go[j].Profit(best_price)-orders_to_go[j].costo_trasporto()):
            orders_to_go[i],orders_to_go[j]=orders_to_go[j],orders_to_go[i]
orders_to_go.reverse()

_ = [otg.Details() for otg in orders_to_go]

Il miglior prezzo è €12.37
Si eseguiranno 75 ordini
Angelica  Calcaterra ha ordinato 486 magliette taglia XS a 12.88€ all'indirizzo [45.420593, 8.805503]
Gianmarco  Calcaterra ha ordinato 436 magliette taglia S a 12.86€ all'indirizzo [45.554086, 8.853124]
Filippo  Fedeli ha ordinato 363 magliette taglia XXL a 19.2€ all'indirizzo [45.318257, 9.186061]
Valentina  Fedeli ha ordinato 425 magliette taglia XL a 13.73€ all'indirizzo [45.629606, 9.662371]
Gianmarco  Fedeli ha ordinato 441 magliette taglia M a 16.77€ all'indirizzo [45.196751, 9.768684]
Davide  Baroni ha ordinato 431 magliette taglia S a 19.62€ all'indirizzo [45.755771, 8.645121]
Francesco  Sartor ha ordinato 318 magliette taglia L a 16.02€ all'indirizzo [45.760935, 9.123703]
Carlo  Augusto ha ordinato 357 magliette taglia XS a 17.5€ all'indirizzo [45.250094, 8.771933]
Maurizio  Baroni ha ordinato 360 magliette taglia M a 17.72€ all'indirizzo [45.069648, 9.03768]
Maurizio  Sartor ha ordinato 471 magliette taglia M a 13.26€ all'i